# Experiment 002: Optimization with fix_direction and Local Search

Since the C++ bbox3 source is truncated, we'll implement Python-based optimization:
1. Apply fix_direction (rotation optimization) to each configuration
2. Run local search to improve boundary trees
3. Validate and score

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import warnings
import time
warnings.filterwarnings('ignore')

getcontext().prec = 30
print('Libraries loaded')

Libraries loaded


In [2]:
# ChristmasTree class
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        
        trunk_w, trunk_h = Decimal('0.15'), Decimal('0.2')
        base_w, mid_w, top_w = Decimal('0.7'), Decimal('0.4'), Decimal('0.25')
        tip_y, tier_1_y, tier_2_y = Decimal('0.8'), Decimal('0.5'), Decimal('0.25')
        
        initial_polygon = Polygon([
            (0, float(tip_y)), (float(top_w/2), float(tier_1_y)), (float(top_w/4), float(tier_1_y)),
            (float(mid_w/2), float(tier_2_y)), (float(mid_w/4), float(tier_2_y)), (float(base_w/2), 0),
            (float(trunk_w/2), 0), (float(trunk_w/2), float(-trunk_h)),
            (float(-trunk_w/2), float(-trunk_h)), (float(-trunk_w/2), 0), (float(-base_w/2), 0),
            (float(-mid_w/4), float(tier_2_y)), (float(-mid_w/2), float(tier_2_y)),
            (float(-top_w/4), float(tier_1_y)), (float(-top_w/2), float(tier_1_y))
        ])
        
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))
    
    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

print('ChristmasTree class defined')

ChristmasTree class defined


In [3]:
# Scoring and utility functions
def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(str(max(bounds[2] - bounds[0], bounds[3] - bounds[1])))

def get_total_score(dict_of_side_length):
    score = Decimal('0')
    for k, v in dict_of_side_length.items():
        score += v ** 2 / Decimal(str(k))
    return score

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                intersection = poly.intersection(polygons[idx])
                if intersection.area > 1e-10:
                    return True
    return False

def load_submission(path):
    df = pd.read_csv(path)
    def parse_value(val):
        if isinstance(val, str) and val.startswith('s'):
            return val[1:]
        return str(val)
    
    df['x'] = df['x'].apply(parse_value)
    df['y'] = df['y'].apply(parse_value)
    df['deg'] = df['deg'].apply(parse_value)
    
    configs = {}
    for _, row in df.iterrows():
        parts = row['id'].split('_')
        n = int(parts[0])
        if n not in configs:
            configs[n] = []
        configs[n].append(ChristmasTree(row['x'], row['y'], row['deg']))
    return configs

print('Utility functions defined')

Utility functions defined


In [4]:
# fix_direction: Rotation optimization
def calculate_bbox_side_at_angle(angle_deg, points):
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    """Find optimal rotation angle to minimize bounding box."""
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points)
    
    try:
        hull_points = points_np[ConvexHull(points_np).vertices]
    except:
        return 0.0, get_tree_list_side_length(trees)
    
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    res = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0.001, 89.999),
        method='bounded'
    )
    
    found_angle = float(res.x)
    found_side = float(res.fun)
    
    if initial_side - found_side > 1e-7:
        return found_angle, Decimal(str(found_side))
    return 0.0, Decimal(str(initial_side))

def apply_rotation(trees, angle_deg):
    """Apply rotation to all trees around their collective center."""
    if not trees or abs(angle_deg) < 1e-12:
        return [t.clone() for t in trees]
    
    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])
    
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])
    
    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center
    
    new_trees = []
    for i, tree in enumerate(trees):
        new_tree = ChristmasTree(
            str(rotated[i, 0]),
            str(rotated[i, 1]),
            str(Decimal(str(tree.angle)) + Decimal(str(angle_deg)))
        )
        new_trees.append(new_tree)
    return new_trees

print('Rotation optimization functions defined')

Rotation optimization functions defined


In [5]:
# Load the baseline submission
configs = load_submission('/home/code/experiments/001_baseline/preoptimized_submission.csv')
print(f'Loaded {len(configs)} configurations')

# Calculate initial score
side_lengths = {}
for n in sorted(configs.keys()):
    side_lengths[n] = get_tree_list_side_length(configs[n])

initial_score = get_total_score(side_lengths)
print(f'Initial score: {float(initial_score):.6f}')

Loaded 200 configurations


Initial score: 70.743774


In [6]:
# Apply fix_direction to all configurations
print('Applying rotation optimization (fix_direction)...')
start_time = time.time()

improved_configs = {}
improved_side_lengths = {}
total_improvement = Decimal('0')

for n in sorted(configs.keys()):
    trees = configs[n]
    old_side = side_lengths[n]
    
    # Find optimal rotation
    best_angle, new_side = optimize_rotation(trees)
    
    if best_angle > 0.001:
        # Apply rotation
        new_trees = apply_rotation(trees, best_angle)
        # Verify no overlaps
        if not has_overlap(new_trees):
            improved_configs[n] = new_trees
            improved_side_lengths[n] = new_side
            improvement = old_side - new_side
            if improvement > Decimal('0.0001'):
                total_improvement += improvement
                if n <= 20 or improvement > Decimal('0.01'):
                    print(f'  n={n}: improved by {float(improvement):.6f} (angle={best_angle:.2f})')
        else:
            improved_configs[n] = trees
            improved_side_lengths[n] = old_side
    else:
        improved_configs[n] = trees
        improved_side_lengths[n] = old_side

elapsed = time.time() - start_time
new_score = get_total_score(improved_side_lengths)
print(f'\nRotation optimization complete in {elapsed:.1f}s')
print(f'Total side length improvement: {float(total_improvement):.6f}')
print(f'New score: {float(new_score):.6f}')
print(f'Score improvement: {float(initial_score - new_score):.6f}')

Applying rotation optimization (fix_direction)...



Rotation optimization complete in 0.5s
Total side length improvement: 0.000000
New score: 70.743774
Score improvement: 0.000000


In [7]:
# Save improved submission
def save_submission(configs, path):
    rows = []
    for n in sorted(configs.keys()):
        for i, tree in enumerate(configs[n]):
            rows.append({
                'id': f'{n:03d}_{i}',
                'x': f's{tree.center_x}',
                'y': f's{tree.center_y}',
                'deg': f's{tree.angle}'
            })
    df = pd.DataFrame(rows)
    df.to_csv(path, index=False)
    print(f'Saved to {path}')

# Save the improved submission
save_submission(improved_configs, '/home/code/experiments/002_bbox3_optimization/optimized_submission.csv')

# Validate no overlaps
print('\nValidating no overlaps...')
overlapping = []
for n in sorted(improved_configs.keys()):
    if has_overlap(improved_configs[n]):
        overlapping.append(n)
        
if overlapping:
    print(f'WARNING: Overlaps found in {len(overlapping)} configs: {overlapping[:10]}')
else:
    print('No overlaps detected!')

Saved to /home/code/experiments/002_bbox3_optimization/optimized_submission.csv

Validating no overlaps...


No overlaps detected!


In [8]:
# Copy to submission folder if improved
import shutil
import json

if float(new_score) < float(initial_score):
    shutil.copy('/home/code/experiments/002_bbox3_optimization/optimized_submission.csv', 
                '/home/submission/submission.csv')
    print(f'Copied improved submission to /home/submission/submission.csv')
    print(f'Score improved from {float(initial_score):.6f} to {float(new_score):.6f}')
else:
    print(f'No improvement. Keeping original submission.')
    shutil.copy('/home/code/experiments/001_baseline/preoptimized_submission.csv',
                '/home/submission/submission.csv')

# Save metrics
with open('/home/code/experiments/002_bbox3_optimization/metrics.json', 'w') as f:
    json.dump({
        'initial_score': float(initial_score),
        'final_score': float(new_score),
        'improvement': float(initial_score - new_score),
        'overlapping_configs': overlapping
    }, f)

print(f'\nFinal Score: {float(new_score):.6f}')

No improvement. Keeping original submission.

Final Score: 70.743774
